In [3]:
import pandas as pd
import glob
import os
import json
from anarci import anarci
from tqdm.notebook import tqdm

In [14]:
def load_asd_data_with_pandas(data_path: str) -> pd.DataFrame:
    """
    Загружает все parquet файлы из папки asd в один pandas DataFrame

    Args:
        data_path: путь к папке с данными

    Returns:
        pd.DataFrame: объединенный DataFrame со всеми данными
    """
    # Получаем все parquet файлы из папки
    parquet_files = glob.glob(os.path.join(data_path, "part-*.parquet"))

    if not parquet_files:
        raise ValueError(f"Не найдено parquet файлов в папке {data_path}")

    print(f"Найдено {len(parquet_files)} parquet файлов")

    # Загружаем все файлы в список DataFrame'ов
    dataframes = []
    for file_path in parquet_files:
        df = pd.read_parquet(file_path)
        dataframes.append(df)

    # Объединяем все DataFrame'ы в один
    combined_df = pd.concat(dataframes, ignore_index=True)

    print(f"Общий размер данных: {combined_df.shape}")
    print(f"Колонки: {list(combined_df.columns)}")

    return combined_df

# Загружаем данные
agab_df = load_asd_data_with_pandas('./asd')

Найдено 20 parquet файлов
Общий размер данных: (1227083, 11)
Колонки: ['dataset', 'heavy_sequence', 'light_sequence', 'scfv', 'affinity_type', 'affinity', 'antigen_sequence', 'confidence', 'nanobody', 'metadata', 'processed_measurement']


In [15]:
# Переводим affinity и processed_measurement во float где возможно
agab_df = agab_df.copy()
agab_df.loc[:, 'affinity'] = pd.to_numeric(agab_df['affinity'], errors='coerce').fillna(agab_df['affinity'])
agab_df.loc[:, 'processed_measurement'] = pd.to_numeric(agab_df['processed_measurement'], errors='coerce').fillna(agab_df['processed_measurement'])
type(agab_df['affinity'][agab_df['affinity_type'] == 'bool'].iloc[0])

float

#### Base filtering

In [16]:
is_not_nanobody = agab_df['nanobody'] == False
# is_high_confidence = agab_df['confidence'].isin(['high', 'very_high'])
is_scfv = agab_df['scfv'] == True
has_both_chains = (
    agab_df['light_sequence'].notna() 
    & agab_df['heavy_sequence'].notna()
    & (agab_df['light_sequence'] != '')
    & (agab_df['heavy_sequence'] != '')
)

agab_df = agab_df[is_not_nanobody 
# & is_high_confidence 
& (is_scfv | has_both_chains)]
agab_df.shape

(844872, 11)

In [5]:
agab_df.groupby(['affinity_type', 'dataset'])['antigen_sequence'].agg([
    ('total_count', 'count'),
    ('unique_antigens', 'nunique')
]).sort_values('unique_antigens', ascending=False)

total_count  unique_antigens
affinity_type         dataset                                              
bool                  patents                        21621             3478
                      structures-antibodies           2711             1083
                      genbank                           98               23
                      skempiv2                          34               21
ddg                   skempiv2                         400               19
elisa_mut_to_wt_ratio abdesign                         658               13
bool                  abdesign                          14               13
                      ab-bind                           13               10
ic_50                 dlgo                             360               10
ddg                   ab-bind                          270                9
-log KD               abbd                          152401                5
kd                    flab_hie2022                      55                3
alphaseq              alphaseq                      131645                2
kd                    flab_rosace2023                   25                2
log_enrichment        abbd                            3452                2
fuzzy                 buzz                          524346                1
kd                    flab_koenig2017                 4275                1
                      flab_shanehsazzadeh2023          446                1
                      flab_warszawski2019             2048                1

In [17]:
agab_df[agab_df['affinity_type'] == 'ddg'].sample(10)

,dataset,heavy_sequence,light_sequence,scfv,affinity_type,affinity,antigen_sequence,confidence,nanobody,metadata,processed_measurement
612041,skempiv2,QVQLVQSGSGVKKPGASVRVSCWTSEDIFERTELIHWVRQAPGQGL...,EIVLTQSPGTLSLSPGETASLSCTAASYGHMTWYQKKPGQPPKLLI...,False,ddg,-0.232216,VWKDADTTLFCASDAKAHETEVHNVWATHACVPTDPNPQEIHLENV...,very_high,False,"{'target_name': '', 'target_pdb': '3se9', 'tar...",-0.232216
981375,skempiv2,QVQLQESGAEVMKPGASVKISCKATGYTFSTYWIEWVKQRPGHGLE...,DIELTQSPATLSVTPGDSVSLSCRASQSISNNLHWYQQKSHESPRL...,False,ddg,-1.253353,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,very_high,False,"{'target_name': '', 'target_pdb': '1mlc', 'tar...",-1.251677
735123,skempiv2,QVQLVQSGAVIKTPGSSVKISCRASGYNFRDYSIHWVRLIPDKGFE...,EIVLTQSPGILSLSPGETATLFCKASQGGNAMTWYQKRRGQVPRLL...,False,ddg,0.0969,VWKDADTTLFCASDAKAHETEVHNVWATHACVPTDPNPQEIHLENV...,very_high,False,"{'target_name': '', 'target_pdb': '3se8', 'tar...",0.0969
1227049,skempiv2,QVQLVQSGGQMKKPGESMRISCRASGYEFIDCTLNWIRLAPGKRPE...,VLTQSPGTLSLSPGETAIISCRTSQYGSLAWYQQRPGQAPRLVIYS...,False,ddg,1.202171,VWKDADTTLFCASDAKAHETEVHNVWATHACVPTDPNPQEIHLENV...,very_high,False,"{'target_name': '', 'target_pdb': '3ngb', 'tar...",1.202171
919231,skempiv2,DVQLQESGPSLVKPSQTLSLTCSVTGDSITADYWSWIRKFPGNRLE...,DIVLTQSPATLSVTPGNSVSLSCRASQSIGNNLHWYQQKSHESPRL...,False,ddg,0.170362,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,very_high,False,"{'target_name': '', 'target_pdb': '3hfm', 'tar...",0.170181
365838,skempiv2,EVQLVESGGGLVQPGGSLRLSCAASGYDFTNYGMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCSASQDISNYLNWYQQKPGKAPKV...,False,ddg,-0.220035,VVKFMDVYQRSYCHPIETLVDIFQEYPDEIEYIFKPSCVPLMRCGG...,very_high,False,"{'target_name': '', 'target_pdb': '1bj1', 'tar...",-0.220035
1227019,skempiv2,DVQLQESGPSLVKPSQTLSLTCSVTGDSITSDYWSWIRKFPGNRLE...,DIVLTQSPATLSVTPGNSVSLSCRASQSIGNNLHWYQQKSHESPRL...,False,ddg,5.510684,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,very_high,False,"{'target_name': '', 'target_pdb': '3hfm', 'tar...",5.510684
304965,ab-bind,EVQLVESGGGLVQPGGSLRLSCAASGFTFSRYTMSWVRQAPGKGLE...,IQLTQSPSSLSASVGDRVTITCSASSSVNHMFWYQQKPGKAPKPWI...,False,ddg,8.0,TQLDIVIVLDGSNSIYPWESVIAFLNDLLKRMDIGPKQTQVGIVQY...,very_high,False,"{'target_name': '', 'target_pdb': '1mhp', 'tar...",8.0
244651,ab-bind,EVQLVESGGGLVQPGGSLRLSCAASGFTFSRYTMSWVRQAPGKGLE...,IQLTQSPSSLSASVGDRVTITCSASSSVNHMFWYQQKPGKAPKPWI...,False,ddg,0.93,TQLDIVIVLDGSNSIYPWESVIAFLNDLLKRMDIGPKQTQVGIVQY...,very_high,False,"{'target_name': '', 'target_pdb': '1mhp', 'tar...",0.884134
919241,ab-bind,EVQLVESGGGLVQPGGSLRLSCAASGFTFSRYTMSWVRQAPGKGLE...,IQLTQSPSSLSASVGDRVTITCSASSSVNHMFWYQQKPGKAPKPWI...,False,ddg,1.36,TQLDIVIVLDGSNSIYPWESVIAFLNDLLKRMDIGPKQTQVGIVQY...,very_high,False,"{'target_name': '', 'target_pdb': '1mhp', 'tar...",1.36178


In [19]:
# (порог, оператор): '<', '>', '=='
AFFINITY_THRESHOLDS = {
    'fuzzy': ('h', '=='),
    'bool': (1, '=='),
    'alphaseq': (2, '<'),
    '-log KD': (7, '>'),
    'kd': (100, '<'),
    'delta_g': (-9.5, '<'),
    'ddg': (0.5, '<'),
    'log_enrichment': (1, '>'),
    'elisa_mut_to_wt_ratio': (1, '>'),
    'ic_50': (100, '<'),
}

def apply_affinity_filter(df: pd.DataFrame, thresholds: dict) -> pd.DataFrame:
    masks = []
    for affinity_type, (threshold, op) in thresholds.items():
        type_mask = df['affinity_type'] == affinity_type
        if op == '==':
            mask = type_mask & (df['processed_measurement'] == threshold)
        else:
            numeric_affinity = pd.to_numeric(df['processed_measurement'], errors='coerce')
            if op == '<':
                mask = type_mask & (numeric_affinity < threshold)
            else:
                mask = type_mask & (numeric_affinity > threshold)
        masks.append(mask)

    if masks:
        combined_mask = masks[0]
        for mask in masks[1:]:
            combined_mask = combined_mask | mask
        return df[combined_mask]
    else:
        return df

print(f"После базовых фильтров: {agab_df.shape}")
print(f"Распределение по affinity_type до фильтрации по порогам:")
print(agab_df['affinity_type'].value_counts())

# Применяем фильтрацию по порогам аффиности
agab_df = apply_affinity_filter(agab_df, AFFINITY_THRESHOLDS)

print(f"\nПосле фильтрации по порогам аффинитета: {agab_df.shape}")
print(f"Распределение по affinity_type после фильтрации:")
print(agab_df['affinity_type'].value_counts())


После базовых фильтров: (772185, 11)
Распределение по affinity_type до фильтрации по порогам:
affinity_type
fuzzy                    524346
alphaseq                 127267
-log KD                   85494
bool                      23277
kd                         6848
log_enrichment             3452
elisa_mut_to_wt_ratio       657
ddg                         484
ic_50                       360
Name: count, dtype: int64

После фильтрации по порогам аффинитета: (337416, 11)
Распределение по affinity_type после фильтрации:
affinity_type
fuzzy                    172149
-log KD                   79889
alphaseq                  53603
bool                      23277
kd                         6743
log_enrichment             1016
ic_50                       349
ddg                         222
elisa_mut_to_wt_ratio       168
Name: count, dtype: int64


In [20]:
agab_df.groupby(['affinity_type', 'dataset'])['antigen_sequence'].agg([
    ('total_count', 'count'),
    ('unique_antigens', 'nunique')
]).sort_values('unique_antigens', ascending=False)

total_count  unique_antigens
affinity_type         dataset                                              
bool                  patents                        21621             3478
                      structures-antibodies           1537             1083
                      genbank                           82               23
                      skempiv2                          22               20
ddg                   skempiv2                         156               19
bool                  abdesign                          13               13
elisa_mut_to_wt_ratio abdesign                         168               12
ic_50                 dlgo                             349               10
-log KD               abbd                           79889                5
ddg                   ab-bind                           66                5
kd                    flab_hie2022                      55                3
alphaseq              alphaseq                       53603                2
bool                  ab-bind                            2                2
kd                    flab_rosace2023                   25                2
fuzzy                 buzz                          172149                1
kd                    flab_koenig2017                 4275                1
                      flab_shanehsazzadeh2023          340                1
                      flab_warszawski2019             2048                1
log_enrichment        abbd                            1016                1

#### Affinity variation

In [64]:
print("=== АНАЛИЗ РАЗЛИЧИЙ В ПОЛЯХ ДЛЯ ДУБЛИКАТОВ ПО ПОСЛЕДОВАТЕЛЬНОСТЯМ (agab_df) ===\n")

# Поля для проверки
fields = [
    "dataset",
    "scfv",
    "affinity_type",
    "affinity",
    "confidence",
    "nanobody",
    "processed_measurement",
]

# --- Поиск групп дубликатов по heavy_sequence + light_sequence + antigen_sequence---
# Находим группы, где хотя бы 2 записи с одинаковой парой последовательностей
group_cols = ["heavy_sequence", "light_sequence", "antigen_sequence"]
grouped = agab_df.groupby(group_cols, sort=False)

# Маска строк, которые входят в группы размером > 1
dup_mask = grouped[group_cols[0]].transform("size") > 1
hl_duplicates = agab_df.loc[dup_mask].copy()

# Переиспользуем groupby только на дубликатах
dup_groups = hl_duplicates.groupby(group_cols, sort=False)
unique_groups_count = dup_groups.ngroups

print(f"Всего групп дубликатов: {unique_groups_count}\n")

# --- Анализ различий по полям ---

# Для каждой группы считаем, сколько уникальных значений в каждом поле
# dropna=False, чтобы различия NaN / не-NaN тоже учитывались
nunique_per_group = dup_groups[fields].nunique(dropna=False)

# Булева матрица: True, если в группе по полю есть различия
diff_mask = nunique_per_group > 1

# Есть ли вообще различия в группе
group_has_diffs = diff_mask.any(axis=1)

groups_with_diffs = int(group_has_diffs.sum())
groups_identical = int((~group_has_diffs).sum())
total = groups_with_diffs + groups_identical if (groups_with_diffs + groups_identical) > 0 else 1

print(f"Групп с различиями: {groups_with_diffs} ({groups_with_diffs / total * 100:.2f}%)")
print(f"Групп без различий: {groups_identical} ({groups_identical / total * 100:.2f}%)\n")

# Количество строк (записей) с различиями и без различий
rows_with_diffs = group_has_diffs[group_has_diffs].index  # индексы групп с различиями
rows_without_diffs = group_has_diffs[~group_has_diffs].index  # индексы групп без различий

num_rows_with_diffs = dup_groups.size().loc[rows_with_diffs].sum() if len(rows_with_diffs) > 0 else 0
num_rows_without_diffs = dup_groups.size().loc[rows_without_diffs].sum() if len(rows_without_diffs) > 0 else 0
rows_total = num_rows_with_diffs + num_rows_without_diffs if (num_rows_with_diffs + num_rows_without_diffs) > 0 else 1

print(f"Строк (записей) с различиями: {num_rows_with_diffs} ({num_rows_with_diffs / rows_total * 100:.2f}%)")
print(f"Строк (записей) без различий: {num_rows_without_diffs} ({num_rows_without_diffs / rows_total * 100:.2f}%)\n")

# Считаем, по скольким группам отличается каждое поле
field_diffs_counts = diff_mask.sum().sort_values(ascending=False)

if groups_with_diffs > 0 and not field_diffs_counts.empty:
    print("=== ПОЛЯ, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===\n")
    for field, count in field_diffs_counts.items():
        if count == 0:
            continue
        print(f"{field}: {count} групп ({count / groups_with_diffs * 100:.2f}%)")

=== АНАЛИЗ РАЗЛИЧИЙ В ПОЛЯХ ДЛЯ ДУБЛИКАТОВ ПО ПОСЛЕДОВАТЕЛЬНОСТЯМ (agab_df) ===

Всего групп дубликатов: 64032

Групп с различиями: 63348 (98.93%)
Групп без различий: 684 (1.07%)

Строк (записей) с различиями: 127356 (98.55%)
Строк (записей) без различий: 1872 (1.45%)

=== ПОЛЯ, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===

affinity: 63326 групп (99.97%)
dataset: 22 групп (0.03%)


In [65]:
print("=== АНАЛИЗ РАЗБРОСА АФФИНИТЕТА В ГРУППАХ ДУБЛИКАТОВ ===\n")

# Работаем с копией дубликатов
analysis_df = hl_duplicates.copy()

# Преобразуем аффинитет в числа, нечисловые значения станут NaN
analysis_df['affinity_numeric'] = pd.to_numeric(analysis_df['affinity'], errors='coerce')

# Группируем
grouped_analysis = analysis_df.groupby(group_cols, sort=False)

# Считаем размах (max - min) и проверяем единственность типа аффинитета
agg_funcs = {
    'affinity_numeric': lambda x: x.max() - x.min(),
    'affinity_type': 'nunique'
}

group_stats = grouped_analysis.agg(agg_funcs)
group_stats.columns = ['affinity_range', 'affinity_type_count']

# Фильтруем группы, где affinity_numeric удалось посчитать (не NaN)
valid_stats = group_stats.dropna(subset=['affinity_range'])

print(f"Групп с валидными числовыми аффинитетами: {len(valid_stats)}")
print(f"Средний разброс аффинитета в группе: {valid_stats['affinity_range'].mean():.4f}")
print(f"Медианный разброс: {valid_stats['affinity_range'].median():.4f}")
print(f"Максимальный разброс: {valid_stats['affinity_range'].max():.4f}\n")

# Проверка на разные типы аффинитета в одной группе
mixed_types = valid_stats[valid_stats['affinity_type_count'] > 1]
if not mixed_types.empty:
    print(f"ВНИМАНИЕ: В {len(mixed_types)} группах смешаны разные типы аффинитета (сравнение может быть некорректным).")
else:
    print("Типы аффинитета внутри каждой группы совпадают (корректно сравнивать числа).\n")

# Топ-5 групп с самым большим разбросом
print("=== ТОП-5 ГРУПП С САМЫМ БОЛЬШИМ РАЗБРОСОМ АФФИНИТЕТА ===")
top_diff_groups = valid_stats.nlargest(5, 'affinity_range')

for idx, (indices, row) in enumerate(top_diff_groups.iterrows()):
    heavy, light, antigen = indices
    print(f"\nГруппа {idx+1} (разброс {row['affinity_range']:.4f}):")
    # Получаем строки этой группы
    group_rows = analysis_df[
        (analysis_df['heavy_sequence'] == heavy) & 
        (analysis_df['light_sequence'] == light) &
        (analysis_df['antigen_sequence'] == antigen)
    ]
    display(group_rows[['dataset', 'affinity_type', 'affinity', 'confidence', 'processed_measurement']])

=== АНАЛИЗ РАЗБРОСА АФФИНИТЕТА В ГРУППАХ ДУБЛИКАТОВ ===

Групп с валидными числовыми аффинитетами: 64032
Средний разброс аффинитета в группе: 2.8970
Медианный разброс: 3.2739
Максимальный разброс: 3.8345

Типы аффинитета внутри каждой группы совпадают (корректно сравнивать числа).

=== ТОП-5 ГРУПП С САМЫМ БОЛЬШИМ РАЗБРОСОМ АФФИНИТЕТА ===

Группа 1 (разброс 3.8345):


,dataset,affinity_type,affinity,confidence,processed_measurement
1021153,abbd,-log KD,9.834508,high,7.917254
1021154,abbd,-log KD,6.0,high,7.917254



Группа 2 (разброс 3.7936):


,dataset,affinity_type,affinity,confidence,processed_measurement
409471,abbd,-log KD,9.793633,high,7.896816
409472,abbd,-log KD,6.0,high,7.896816



Группа 3 (разброс 3.7556):


,dataset,affinity_type,affinity,confidence,processed_measurement
471033,abbd,-log KD,9.755628,high,7.877814
471034,abbd,-log KD,6.0,high,7.877814



Группа 4 (разброс 3.7512):


,dataset,affinity_type,affinity,confidence,processed_measurement
591881,abbd,-log KD,9.75118,high,7.87559
591882,abbd,-log KD,6.0,high,7.87559



Группа 5 (разброс 3.7379):


,dataset,affinity_type,affinity,confidence,processed_measurement
961707,abbd,-log KD,9.737911,high,7.868956
961708,abbd,-log KD,6.0,high,7.868956


#### Drop duplicates

In [21]:
# Удаляем дубликаты по heavy_sequence + light_sequence + antigen_sequence
# Оставляем первую запись из каждой группы (affinity отличается, но это не важно)

print(f"Размер до удаления дубликатов: {agab_df.shape}")

agab_df = agab_df.drop_duplicates(
    subset=['heavy_sequence', 'light_sequence', 'antigen_sequence'],
    keep='first'
)

print(f"Размер после удаления дубликатов: {agab_df.shape}")

Размер до удаления дубликатов: (337416, 11)
Размер после удаления дубликатов: (337416, 11)


#### ANARCI

In [22]:
def is_empty(x):
    return x is None or (isinstance(x, float) and np.isnan(x)) or (isinstance(x, str) and x.strip() == "")

scfv_df = agab_df[agab_df['scfv'] == True].copy()

# сколько scFv-строк уже имеют обе цепи
both_present = (~scfv_df["heavy_sequence"].apply(is_empty)) & (~scfv_df["light_sequence"].apply(is_empty))
only_heavy   = (~scfv_df["heavy_sequence"].apply(is_empty)) & ( scfv_df["light_sequence"].apply(is_empty))
only_light   = ( scfv_df["heavy_sequence"].apply(is_empty)) & (~scfv_df["light_sequence"].apply(is_empty))

print("scFv всего:", len(scfv_df))
print("оба домена уже раздельно:", both_present.sum())
print("только heavy заполнен:", only_heavy.sum())
print("только light заполнен:", only_light.sum())
print("оба пустые:", ((~both_present) & (~only_heavy) & (~only_light)).sum())

scFv всего: 53603
оба домена уже раздельно: 0
только heavy заполнен: 53603
только light заполнен: 0
оба пустые: 0


In [ ]:
from multiprocessing import Pool
import numpy as np
import sys
import os

# Add current directory to path to import worker module
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Import worker function from external file to fix pickling error
try:
    from data.anarci_worker import run_anarci_batch
except ImportError:
    # Fallback if running from data directory directly
    try:
        from anarci_worker import run_anarci_batch
    except ImportError:
        print("Error: Could not import anarci_worker. Make sure data/anarci_worker.py exists.")

def process_antibody_sequences_optimized(df, n_jobs=8, batch_size=5000):
    """
    Optimized version of antibody processing using multiprocessing.
    Runs ANARCI in parallel batches and minimizes DataFrame operations.
    """
    df = df.copy()
    
    # 1. Prepare sequences list efficiently
    print("Preparing sequences...")
    sequences_to_anarci = []
    mapping_list = [] 
    
    rows_iter = zip(df.index, df['scfv'], df['heavy_sequence'], df['light_sequence'])
    
    for idx, is_scfv, h_seq, l_seq in tqdm(rows_iter, total=len(df), desc="Collecting sequences"):
        if is_scfv:
            seq = h_seq
            if pd.isna(seq) or len(seq) == 0:
                 seq = l_seq if isinstance(l_seq, str) else ''
            
            if isinstance(seq, str) and len(seq) > 0:
                internal_id = f"{idx}_scfv"
                sequences_to_anarci.append((internal_id, seq))
                mapping_list.append((idx, 'scfv', seq))
        else:
            if isinstance(h_seq, str) and len(h_seq) > 0:
                internal_id = f"{idx}_heavy"
                sequences_to_anarci.append((internal_id, h_seq))
                mapping_list.append((idx, 'heavy', h_seq))
            
            if isinstance(l_seq, str) and len(l_seq) > 0:
                internal_id = f"{idx}_light"
                sequences_to_anarci.append((internal_id, l_seq))
                mapping_list.append((idx, 'light', l_seq))
                
    total_seqs = len(sequences_to_anarci)
    print(f"Running ANARCI on {total_seqs} sequences using {n_jobs} processes...")
    
    # 2. Parallel execution
    chunk_size = min(batch_size, max(1, total_seqs // (n_jobs * 2)))
    chunks = [sequences_to_anarci[i : i + chunk_size] for i in range(0, total_seqs, chunk_size)]
    
    all_numbering = []
    all_alignment_details = []
    
    with Pool(processes=n_jobs) as pool:
        results = list(tqdm(pool.imap(run_anarci_batch, chunks), total=len(chunks), desc="ANARCI Parallel"))
    
    # Unpack results
    for num, aln, err in results:
        if err and not isinstance(err, str): 
             pass
        if num is None:
            print(f"Batch failed: {err}")
            continue 
        all_numbering.extend(num)
        all_alignment_details.extend(aln)

    # 3. Process results efficiently
    print("Processing ANARCI results...")
    
    heavy_seqs = {}
    light_seqs = {}
    heavy_specs = {}
    light_specs = {}
    heavy_germs = {}
    light_germs = {}
    heavy_nums = {}
    light_nums = {}
    
    for i, (num_hits, hits) in tqdm(enumerate(zip(all_numbering, all_alignment_details)), total=len(all_numbering), desc="Parsing results"):
        if not hits or not num_hits: 
            continue
            
        original_idx, seq_type, full_seq = mapping_list[i]
        
        for domain_idx, hit in enumerate(hits):
            if not hit: continue
            
            chain_type = hit.get('chain_type', 'unknown')
            start = hit.get('query_start')
            end = hit.get('query_end')
            species = hit.get('species')
            germlines = hit.get('germlines')
            
            domain_data = num_hits[domain_idx]
            numbering_json = None
            if domain_data:
                residues_json = [
                    {"pos": r[0][0], "ins": r[0][1].strip(), "aa": r[1]}
                    for r in domain_data[0]
                ]
                numbering_obj = {
                    "domain_start": domain_data[1],
                    "domain_end": domain_data[2],
                    "residues": residues_json
                }
                numbering_json = json.dumps([numbering_obj])
            
            domain_seq = full_seq[start : end] if (start is not None and end is not None) else None

            is_heavy = False
            if seq_type == 'scfv':
                if chain_type == 'H': is_heavy = True
                elif chain_type in ['K', 'L']: is_heavy = False
                else: continue
            elif seq_type == 'heavy':
                if chain_type == 'H': is_heavy = True
                else: continue
            elif seq_type == 'light':
                if chain_type in ['K', 'L']: is_heavy = False
                else: continue
            
            if is_heavy:
                heavy_seqs[original_idx] = domain_seq
                heavy_specs[original_idx] = species
                heavy_germs[original_idx] = str(germlines)
                heavy_nums[original_idx] = numbering_json
            else:
                light_seqs[original_idx] = domain_seq
                light_specs[original_idx] = species
                light_germs[original_idx] = str(germlines)
                light_nums[original_idx] = numbering_json

    print("Updating DataFrame columns...")
    
    # Convert dictionaries to Series once
    heavy_seqs_series = pd.Series(heavy_seqs, name='heavy_sequence')
    light_seqs_series = pd.Series(light_seqs, name='light_sequence')
    
    # Use combine_first to update only found values
    # heavy_seqs_series contains only updated sequences, combine_first fills gaps from original
    df['heavy_sequence'] = heavy_seqs_series.combine_first(df['heavy_sequence'])
    df['light_sequence'] = light_seqs_series.combine_first(df['light_sequence'])
    
    # For new columns, map is fine as they are empty or overwritten
    df['heavy_species'] = df.index.map(heavy_specs)
    df['light_species'] = df.index.map(light_specs)
    df['heavy_germlines'] = df.index.map(heavy_germs)
    df['light_germlines'] = df.index.map(light_germs)
    df['heavy_numbering'] = df.index.map(heavy_nums)
    df['light_numbering'] = df.index.map(light_nums)
    found_h = df['heavy_sequence'].notna().sum()
    found_l = df['light_sequence'].notna().sum()
    print(f"Extraction complete. Found {found_h} Heavy chains and {found_l} Light chains.")
    
    return df

# Apply the optimized function
if __name__ == '__main__':
    agab_df = process_antibody_sequences_optimized(agab_df, n_jobs=16)


# Сохраняем датафрейм в .parquet
output_path = 'agab_after_anarci.parquet'
df = agab_df

# Fix mixed types for Parquet export
# 'affinity' contains both numbers and strings (e.g. 'h'), so we must save as string
df['affinity'] = df['affinity'].astype(str)
df['processed_measurement'] = df['processed_measurement'].astype(str)

df.to_parquet(output_path, index=False, engine='pyarrow')
print(f"Отфильтрованные данные сохранены в {output_path}")
print(f"Размер сохраненных данных: {df.shape}")


Preparing sequences...


Running ANARCI on 620789 sequences using 16 processes...


ANARCI Parallel:   0%|          | 0/125 [00:00<?, ?it/s]

Processing ANARCI results...


Parsing results:   0%|          | 0/620789 [00:00<?, ?it/s]

Updating DataFrame columns...
Extraction complete. Found 337196 Heavy chains and 337196 Light chains.


### Фильтрация после ANARCI разметки


Ответ на вопрос про антитела > 140 а.к.
Стоит ли их оставлять?
Да, однозначно стоит, но с проверкой.
Длинные последовательности (>140 а.к.) в большинстве случаев — это не ошибка, а антитела с ультра-длинным CDRH3 (петлей, связывающей антиген). Такие антитела встречаются в природе (например, у ВИЧ-инфицированных пациентов или у некоторых видов животных, таких как коровы) и могут обладать уникальными свойствами нейтрализации.
Если вы удалите их просто по порогу длины, вы потеряете самые интересные и редкие данные. Проверка совокупной длины CDR — отличная идея, чтобы убедиться, что "лишняя" длина находится именно в вариабельных петлях (где ей и место), а не в каркасных регионах (что было бы ошибкой секвенирования).

In [ ]:
agab_df = pd.read_parquet('agab_after_anarci.parquet')
print(f"Всего записей: {agab_df.shape}")

Всего записей: (337196, 17)


CDR3 <= 37: "У тебя не может быть одной петли длиной в километр".  

total_cdr_len: "Если ты в целом длинный гигант, докажи это длиной своих петель, а не ошибкой в каркасе".

In [15]:
import pandas as pd
import json

# --- Функции --- (те же самые, без изменений)
def has_conservative_cysteines(numbering_json):
    if not numbering_json or pd.isna(numbering_json):
        return False
    try:
        data = json.loads(numbering_json)
        if not data: return False
        residues = data[0].get('residues', [])
        has_c23 = any(r['pos'] == 23 and r['aa'] == 'C' for r in residues)
        has_c104 = any(r['pos'] == 104 and r['aa'] == 'C' for r in residues)
        return has_c23 and has_c104
    except: return False

def check_cdr3_length(numbering_json, threshold=37):
    if not numbering_json or pd.isna(numbering_json): return False
    try:
        data = json.loads(numbering_json)
        if not data: return False
        residues = data[0].get('residues', [])
        cdr3_len = sum(1 for r in residues if 105 <= r['pos'] <= 117)
        return cdr3_len <= threshold
    except: return False

def get_total_cdr_length(numbering_json):
    if not numbering_json or pd.isna(numbering_json): return 0
    try:
        data = json.loads(numbering_json)
        if not data: return 0
        residues = data[0].get('residues', [])
        cdr_len = 0
        for r in residues:
            p = int(r['pos'])
            if (27 <= p <= 38) or (56 <= p <= 65) or (105 <= p <= 117):
                cdr_len += 1
        return cdr_len
    except: return 0

# === ПРИМЕНЕНИЕ ФИЛЬТРОВ ===

print(f"ИСХОДНЫЙ размер датасета: {len(agab_df)}")
current_df = agab_df.copy()

# --- ШАГ 1: Фильтрация по цистеинам (C23 & C104) ---
print("\n[Шаг 1/3] Проверка консервативных цистеинов (C23 + C104)...")
if 'heavy_cys_ok' not in current_df.columns:
    current_df['heavy_cys_ok'] = current_df['heavy_numbering'].apply(has_conservative_cysteines)
    current_df['light_cys_ok'] = current_df['light_numbering'].apply(has_conservative_cysteines)

h_cys = current_df['heavy_sequence'].isna() | current_df['heavy_cys_ok']
l_cys = current_df['light_sequence'].isna() | current_df['light_cys_ok']
not_empty = current_df['heavy_sequence'].notna() | current_df['light_sequence'].notna()

before_len = len(current_df)
current_df = current_df[h_cys & l_cys & not_empty]
removed_cys = before_len - len(current_df)

print(f"   -> Осталось: {len(current_df)}")
print(f"   -> Удалено: {removed_cys} (отсутствуют критические цистеины)")


# --- ШАГ 2: Фильтрация по длине CDR3 (<= 37 а.к.) ---
print("\n[Шаг 2/3] Проверка длины CDR3 (порог OAS: <= 37 а.к.)...")
current_df['heavy_cdr3_ok'] = current_df['heavy_numbering'].apply(check_cdr3_length)
current_df['light_cdr3_ok'] = current_df['light_numbering'].apply(check_cdr3_length)

h_cdr3 = current_df['heavy_sequence'].isna() | current_df['heavy_cdr3_ok']
l_cdr3 = current_df['light_sequence'].isna() | current_df['light_cdr3_ok']

before_len = len(current_df)
current_df = current_df[h_cdr3 & l_cdr3]
removed_cdr3 = before_len - len(current_df)

print(f"   -> Осталось: {len(current_df)}")
print(f"   -> Удалено: {removed_cdr3} (CDR3 > 37 аминокислот)")


# --- ШАГ 3: Проверка длинных последовательностей (>140 а.к.) ---
print("\n[Шаг 3/3] Анализ сверхдлинных цепей (>140 а.к.)...")
current_df['heavy_len'] = current_df['heavy_sequence'].str.len().fillna(0)
long_mask = current_df['heavy_len'] > 140

long_count = long_mask.sum()
if long_count > 0:
    print(f"   Найдено длинных цепей (>140): {long_count}")
    
    # Считаем сумму длин CDR только для длинных цепей
    current_df.loc[long_mask, 'total_cdr_len'] = current_df.loc[long_mask, 'heavy_numbering'].apply(get_total_cdr_length)
    
    # Условие удаления: Длинная цепь (>140) НО короткие CDR (<35)
    # Это значит, что длина набрана за счет каркаса (ошибка)
    suspicious_mask = long_mask & (current_df['total_cdr_len'] < 35)
    suspicious_count = suspicious_mask.sum()
    
    if suspicious_count > 0:
        before_len = len(current_df)
        current_df = current_df[~suspicious_mask]
        print(f"   -> Удалено: {suspicious_count} (длинные цепи с подозрительно короткими CDR)")
    else:
        print("   -> Удалено: 0 (все длинные цепи имеют адекватно длинные CDR)")
else:
    print("   -> Длинных цепей (>140 а.к.) не обнаружено. Пропуск шага.")

# Удаляем технические колонки перед сохранением
cols_to_drop = ['heavy_cys_ok', 'light_cys_ok', 'heavy_cdr3_ok', 'light_cdr3_ok', 'total_cdr_len', 'heavy_len']
agab_filtered = current_df.drop(columns=cols_to_drop, errors='ignore').copy()

print(f"\n=== ИТОГ ===")
print(f"Финальный размер: {len(agab_filtered)}")
print(f"Всего удалено: {len(agab_df) - len(agab_filtered)}")

ИСХОДНЫЙ размер датасета: 337196

[Шаг 1/3] Проверка консервативных цистеинов (C23 + C104)...
   -> Осталось: 336751
   -> Удалено: 445 (отсутствуют критические цистеины)

[Шаг 2/3] Проверка длины CDR3 (порог OAS: <= 37 а.к.)...
   -> Осталось: 336741
   -> Удалено: 10 (CDR3 > 37 аминокислот)

[Шаг 3/3] Анализ сверхдлинных цепей (>140 а.к.)...
   -> Длинных цепей (>140 а.к.) не обнаружено. Пропуск шага.

=== ИТОГ ===
Финальный размер: 336741
Всего удалено: 455


### Save to agab.parquet

In [16]:
# Сохраняем датафрейм в .parquet
output_path = 'agab.parquet'
df = agab_filtered

# Fix mixed types for Parquet export
# 'affinity' contains both numbers and strings (e.g. 'h'), so we must save as string
df['affinity'] = df['affinity'].astype(str)
df['processed_measurement'] = df['processed_measurement'].astype(str)

df.to_parquet(output_path, index=False, engine='pyarrow')
print(f"Отфильтрованные данные сохранены в {output_path}")
print(f"Размер сохраненных данных: {df.shape}")


Отфильтрованные данные сохранены в agab.parquet
Размер сохраненных данных: (336741, 17)


### Делим на train test

In [6]:
import pandas as pd

# Загружаем из agab_mmseq.parquet
agab_mmseq = pd.read_parquet('agab_mmseq.parquet', engine='pyarrow')
print(f"Загружено: {agab_mmseq.shape}")

Загружено: (336741, 18)


In [7]:
agab_mmseq.head()

,dataset,heavy_sequence,light_sequence,scfv,affinity_type,affinity,antigen_sequence,confidence,nanobody,metadata,processed_measurement,heavy_species,light_species,heavy_germlines,light_germlines,heavy_numbering,light_numbering,mmseq_cluster_rep
0,abdesign,QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYWINWVRQAPGQGLE...,DIVMTQSPDSLAVSLGERATINCKSSQSLWDSGNQKNFLTWYQQKP...,False,elisa_mut_to_wt_ratio,1.008535785,WNPPTFSPALLVVTEGDNATFTCSFSNTSESFVLNWYRMSPSNQTD...,very_high,False,{'heavy_riot_numbering': {'cdr1_aa': 'GYTFTSYW...,1.008535785,mouse,human,"{'v_gene': [('human', 'IGHV1-46*01'), 0.857142...","{'v_gene': [('human', 'IGKV4-1*01'), 0.8877551...","[{""domain_start"": 0, ""domain_end"": 116, ""resid...","[{""domain_start"": 0, ""domain_end"": 112, ""resid...",HHHHHHGENLYFQGSLDSPDRPWNPPTFSPALLVVTEGDNATFTCS...
1,abdesign,VQLVQSGVEVKKPGASVKVSCKASGYTFTNYYMYWVRQAPGQGLEW...,EIVLTQSPATLSLSPGERATLSCRASKGVSTSGYSYLHWYQQKPGQ...,False,elisa_mut_to_wt_ratio,1.052205794,PWNPPTFSPALLVVTEGDNATFTCSFSNTSESFVLNWYRMSPSNQT...,very_high,False,{'heavy_riot_numbering': {'cdr1_aa': 'GYTFTNYY...,1.052205794,mouse,human,"{'v_gene': [('human', 'IGHV1-2*02'), 0.7857142...","{'v_gene': [('human', 'IGKV3-11*01'), 0.858695...","[{""domain_start"": 0, ""domain_end"": 118, ""resid...","[{""domain_start"": 0, ""domain_end"": 110, ""resid...",HHHHHHGENLYFQGSLDSPDRPWNPPTFSPALLVVTEGDNATFTCS...
2,structures-antibodies,QVQLQQPGAELVKPGASVKLSCKASGYTFTSDWIHWVKQRPGHGLE...,DILLTQSPAILSVSPGERVSFSCRASQSIGTDIHWYQQRTNGSPRL...,False,bool,1.0,SALHWRAAGAATVLLVIVLLAGSYLAVLAERGAPGAQLITYPRALW...,very_high,False,{'heavy_riot_numbering': {'cdr1_aa': 'GYTFTSDW...,1.0,mouse,mouse,"{'v_gene': [('mouse', 'IGHV1-69-5*01'), 0.8877...","{'v_gene': [('mouse', 'IGKV5-48*01'), 0.978260...","[{""domain_start"": 0, ""domain_end"": 117, ""resid...","[{""domain_start"": 0, ""domain_end"": 106, ""resid...",MAPMLSGLLARLVKLLLGRHGSALHWRAAGAATVLLVIVLLAGSYL...
3,structures-antibodies,EVQLVESGGGLVQPGGSLRLSCAASGFTFDDYSMNWVRQAPGKGLE...,DIQLTQSPSSLSASVGDRVTITCQASQDISNYLNWYQQKPGKAPKL...,False,bool,1.0,FKVATPYSLYVCPEGQNVTLTCRLLGPVDKGHDVTFYKTWYRSSRG...,very_high,False,{'heavy_riot_numbering': {'cdr1_aa': 'GFTFDDYS...,1.0,human,human,"{'v_gene': [('human', 'IGHV3-48*01'), 0.948979...","{'v_gene': [('human', 'IGKV1-33*01'), 0.967391...","[{""domain_start"": 0, ""domain_end"": 121, ""resid...","[{""domain_start"": 0, ""domain_end"": 107, ""resid...",FKVATPYSLYVCPEGQNVTLTCRLLGPVDKGHDVTFYKTWYRSSRG...
4,structures-antibodies,VKLQESGAVVQPGGSLRLSCAASGFTGSDYDMSWIRQAPGKGLEWV...,DIQMTQSPASLAVSPGQRATITCRASESVSNYGINFINWFQQKPGQ...,False,bool,1.0,ADPGYLLEFDTLCIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDKH...,very_high,False,{'heavy_riot_numbering': {'cdr1_aa': 'GFTGSDYD...,1.0,human,mouse,"{'v_gene': [('human', 'IGHV3-66*01'), 0.773195...","{'v_gene': [('mouse', 'IGKV3-2*01'), 0.8020833...","[{""domain_start"": 0, ""domain_end"": 119, ""resid...","[{""domain_start"": 0, ""domain_end"": 110, ""resid...",MEEIVLLFAIVSLARSDQICIGYHANNSTKQVDTIMEKNVTVTHAQ...


In [ ]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

df = pd.read_parquet('data/agab_mmseq.parquet')

# Используем GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Группируем по представителю кластера!
splitter = gss.split(df, groups=df['mmseq_cluster_rep'])
train_idx, test_idx = next(splitter)

train_df = df.iloc[train_idx]
test_df = df.iloc[test_idx]

print(f"Train size: {len(train_df)}")
print(f"Test size: {len(test_df)}")

# Проверка на утечки (должно быть 0 пересечений по кластерам)
train_clusters = set(train_df['mmseq_cluster_rep'])
test_clusters = set(test_df['mmseq_cluster_rep'])
print(f"Пересечение кластеров: {len(train_clusters.intersection(test_clusters))}") 
# Если 0 — значит, сплит идеальный.